In [1]:
from bs4 import BeautifulSoup
import urllib.request
import string
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity 
import numpy as np
import pickle
import itertools
import pandas as pd

In [2]:
# scrap dari digilib

In [2]:
with open('paper_ori.pkl', 'rb') as f:
    paper = pickle.load(f)

In [3]:
len(paper)

360

In [10]:
# preprocessing

In [10]:
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()
stemmer = StemmerFactory().create_stemmer()

In [4]:
with open('processed_paper.pkl', 'rb') as f:
    processed_paper = pickle.load(f)

In [15]:
# generate thesaurus

In [5]:
with open('thesaurus.pkl', 'rb') as f:
    thesaurus = pickle.load(f)

In [22]:
words = [x for x in thesaurus]

In [81]:
# input query

In [72]:
init_query = 'pengolahan dokumen'

In [51]:
# tf_idf

In [73]:
vectorizer = TfidfVectorizer(use_idf=True)
query = init_query
query = query.lower()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)
query = query.translate(remove_punctuation_map)
query = stopword.remove(query)
query = query.split()
query = [stemmer.stem(x) for x in query]
query

['olah', 'dokumen']

In [74]:
max_result = []
x = [' '.join(query)]
paper_tfidf = vectorizer.fit_transform(x + processed_paper)
q = paper_tfidf[0]
result = cosine_similarity(paper_tfidf, q)
idx = np.argsort(-result,axis=0).flatten()    
final = [[num, y[0], x] for num, y in enumerate(result) if y[0] > 0.0]
max_result += final
max_result = sorted(max_result, key=lambda x: x[1], reverse=True)
set_result = set()
new_result = []
for item in max_result:
    if item[0] not in set_result:
        set_result.add(item[0])
        new_result.append(item)
    else:
        pass
len(new_result)

41

In [75]:
for x in new_result[1:5]: 
    print('Result', x[0]) 
    print('QUERY', x[2]) 
    print(paper[x[0]-1][1]) 
    print(paper[x[0]-1][2][:200] + '...')
    print()

Result 14
QUERY ['olah dokumen']
RANCANG BANGUN SISTEM PELACAKAN DOKUMEN MEMANFAATKAN FUSE DAN SAMBA FILE-SYSTEM STUDI KASUS PT. ANEKA TUNA INDONESIA
Saat ini dokumen digital merupakan aspek penting dalam
dunia bisnis. Dokumen digital memuat data-data penting
perusahaan baik berupa data produksi sampai data keuangan.
Dokumen digital dalam perusa...

Result 15
QUERY ['olah dokumen']
DESIGN OF DOCUMENT TRACKING SYSTEM UTILIZING FUSE AND SAMBA FILE-SYSTEM CASE STUDY IN PT. ANEKA TUNA INDONESIA
Saat ini dokumen digital merupakan aspek penting dalam
dunia bisnis. Dokumen digital memuat data-data penting
perusahaan baik berupa data produksi sampai data keuangan.
Dokumen digital dalam perusa...

Result 32
QUERY ['olah dokumen']
RANCANG BANGUN SISTEM PELACAKAN DOKUMEN MEMANFAATKAN FUSE DAN SAMBA FILE-SYSTEM STUDI KASUS PT. ANEKA TUNA INDONESIA
Saat ini dokumen digital merupakan aspek penting dalam
dunia bisnis. Dokumen digital memuat data-data penting
perusahaan baik berupa data produksi sampa

In [76]:
idx_res = [x[0]-1 for x in new_result[1:]]
file = []
for i,x in enumerate(paper):
    if i in idx_res:
        file.append([x[1],x[2],'ok_ok'])
    else:
        file.append([x[1],x[2],''])
df = pd.DataFrame(file)
df.to_excel('hasil/hasil ori ' +init_query+ '.xlsx', header=False, index=False)

In [66]:
# query_expansion

In [77]:
vectorizer = TfidfVectorizer(use_idf=True)
query = init_query
query = query.lower()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)
query = query.translate(remove_punctuation_map)
query = stopword.remove(query)
query = query.split()
query = [stemmer.stem(x) for x in query]
query

['olah', 'dokumen']

In [78]:
product_query = []
list_synonym = []
for x in query:
    if x in words:
        list_synonym.append(thesaurus[x])
    else:
        name = x
        data = { "q": name }
        encoded_data = urllib.parse.urlencode(data).encode("utf-8")
        content = urllib.request.urlopen("http://www.sinonimkata.com/search.php", encoded_data)
        soup = BeautifulSoup(content, 'html.parser')
        try:
            synonym = soup.find('td', attrs={'width': '90%'}).find_all('a')
            synonym = [x.getText() for x in synonym]
            thesaurus[x] = [x] + synonym
            list_synonym.append(thesaurus[x])
        except:
            list_synonym.append([x])
qs = []
for x in itertools.product(*list_synonym):
    x = [stemmer.stem(y) for y in x]
    qs.append([' '.join(x)])
for x in qs:
    print(x, end=' ')

['olah dokumen'] ['olah akta'] ['olah arsip'] ['olah inskripsi'] ['olah kopi'] ['olah manuskrip'] ['olah tinggal'] ['olah piagam'] ['olah sahifah'] ['olah salin'] ['olah sertifikat'] ['olah surat'] ['olah tembus'] ['olah tindas'] ['akal dokumen'] ['akal akta'] ['akal arsip'] ['akal inskripsi'] ['akal kopi'] ['akal manuskrip'] ['akal tinggal'] ['akal piagam'] ['akal sahifah'] ['akal salin'] ['akal sertifikat'] ['akal surat'] ['akal tembus'] ['akal tindas'] ['canda dokumen'] ['canda akta'] ['canda arsip'] ['canda inskripsi'] ['canda kopi'] ['canda manuskrip'] ['canda tinggal'] ['canda piagam'] ['canda sahifah'] ['canda salin'] ['canda sertifikat'] ['canda surat'] ['canda tembus'] ['canda tindas'] ['cara dokumen'] ['cara akta'] ['cara arsip'] ['cara inskripsi'] ['cara kopi'] ['cara manuskrip'] ['cara tinggal'] ['cara piagam'] ['cara sahifah'] ['cara salin'] ['cara sertifikat'] ['cara surat'] ['cara tembus'] ['cara tindas'] ['daya dokumen'] ['daya akta'] ['daya arsip'] ['daya inskripsi'] [

In [79]:
max_result = []
for x in qs:
    paper_tfidf = vectorizer.fit_transform(x + processed_paper)
    q = paper_tfidf[0]
    result = cosine_similarity(paper_tfidf, q)
    idx = np.argsort(-result,axis=0).flatten()    
    final = [[num, y[0], x] for num, y in enumerate(result) if y[0] > 0.0]
    max_result += final
max_result = sorted(max_result, key=lambda x: x[1], reverse=True)
set_result = set()
new_result = []
for item in max_result:
    if item[0] not in set_result:
        set_result.add(item[0])
        new_result.append(item)
    else:
        pass
len(new_result)

241

In [80]:
for x in new_result[1:5]: 
    print('Result', x[0]+1) 
    print('QUERY', x[2]) 
    print(paper[x[0]-1][1]) 
    print(paper[x[0]-1][2][:200] + '...')
    print()

Result 15
QUERY ['laku dokumen']
RANCANG BANGUN SISTEM PELACAKAN DOKUMEN MEMANFAATKAN FUSE DAN SAMBA FILE-SYSTEM STUDI KASUS PT. ANEKA TUNA INDONESIA
Saat ini dokumen digital merupakan aspek penting dalam
dunia bisnis. Dokumen digital memuat data-data penting
perusahaan baik berupa data produksi sampai data keuangan.
Dokumen digital dalam perusa...

Result 16
QUERY ['laku dokumen']
DESIGN OF DOCUMENT TRACKING SYSTEM UTILIZING FUSE AND SAMBA FILE-SYSTEM CASE STUDY IN PT. ANEKA TUNA INDONESIA
Saat ini dokumen digital merupakan aspek penting dalam
dunia bisnis. Dokumen digital memuat data-data penting
perusahaan baik berupa data produksi sampai data keuangan.
Dokumen digital dalam perusa...

Result 33
QUERY ['laku dokumen']
RANCANG BANGUN SISTEM PELACAKAN DOKUMEN MEMANFAATKAN FUSE DAN SAMBA FILE-SYSTEM STUDI KASUS PT. ANEKA TUNA INDONESIA
Saat ini dokumen digital merupakan aspek penting dalam
dunia bisnis. Dokumen digital memuat data-data penting
perusahaan baik berupa data produksi sampa

In [81]:
idx_res = [x[0]-1 for x in new_result[1:]]
file = []
for i,x in enumerate(paper):
    if i in idx_res:
        file.append([x[1],x[2],'ok_ok'])
    else:
        file.append([x[1],x[2],''])
df = pd.DataFrame(file)
df.to_excel('hasil/hasil expansion ' +init_query+ '.xlsx', header=False, index=False)